Auteur: Michaël Leroy

Récupération des POI tourisme sur data.gouv.fr

    - lien: https://www.data.gouv.fr/fr/datasets/datatourisme-la-base-nationale-des-donnees-publiques-dinformation-touristique-en-open-data/#/resources

    - Classification transformer zero shot classification pour regrouper les POIs 
        -382 types différents en 7 classes
    - Modèle Bart HuggingFace ( il faut ajouter pytorch et transformers à l'env)

[+] Méthodo:

      * le dataset fourni 2 champs exploitables:

            1- Leur ref interne (contenue dans le champ url), 326 tags différents
            2- Une description texte de l'activité

      * Classification sur les tags:
            Par itérations successives ajout d'items aux labels pour guider le regroupement, pas élégant mais efficace.

      * J'ai fait un essai sur la description, traduction en anglais de la phrase + classif: marrant mais clairement il faut du compute pour passer les 338 000 tags!
            -> trouver un zeroshot classifier entrainé sur un corpus francophone   


 Faire attention il y a 2 tags concernant la mobilité électrique:
        pour les voitures
        pour les bicycles... apparament ça recherge beaucoup dans les régions montagneuses.

In [1]:
class CFG:
    area = [
        'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]
    area_name = 'france'      #'_'.join(area)

    

    bornes = False

    viz = True

    roads = (
        '|motorway'
        '|motorway_link'
        '|trunk'
        '|trunk_link'
        '|primary'
        '|primary_link'
        '|secondary'
        '|secondary_link'
        '|tertiary'
         '|tertiary_link'
    )

        # '|primary'
        # '|motorway_link'
        
        # '|primary_link'

        # '|secondary'
        # '|secondary_link'
        
        
        #  '|tertiary'
        #  '|tertiary_link'

    data_path = './osm_datas/'

import joblib
joblib.dump(CFG.area, 'CFG.pkl')    

import os
os.environ['USE_PYGEOS'] = '0'

%load_ext autoreload
%autoreload


%aimport OSM_helper_functions
%aimport OCM_helper_functions

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import osmnx as ox
import networkx as nx
from tqdm.notebook import tqdm
from OSM_helper_functions import *
from OCM_helper_functions import *


ox.settings.log_console=True
ox.settings.use_cache=True
''' To retrieve http data from OSMnx and cache them locally
    before building graph. Raise and exception if requests
    are all done. intercept then set to False and retry.'''
# ox.settings.cache_only_mode=True 

Administratives = {
    'Auvergne-Rhône-Alpes': ['Ain', 'Allier', 'Ardèche', 'Cantal', 'Drôme', 'Isère', 'Loire', 'Haute-Loire', 'Puy-de-Dôme', 'Rhône', 'Savoie', 'Haute-Savoie'],
    'Bourgogne-Franche-Comté': ['Côte-d\'Or', 'Doubs', 'Jura', 'Nièvre', 'Haute-Saône', 'Saône-et-Loire', 'Yonne', 'Territoire de Belfort'],
    'Brittany': ['Côtes-d\'Armor', 'Finistère', 'Ille-et-Vilaine', 'Morbihan'],
    'Centre-Val de Loire': ['Cher', 'Eure-et-Loir', 'Indre', 'Indre-et-Loire', 'Loir-et-Cher', 'Loiret'],
    # 'Corse': ['Corse-du-Sud', 'Haute-Corse'],
    'Grand Est': ['Ardennes', 'Aube', 'Marne', 'Haute-Marne', 'Meurthe-et-Moselle', 'Meuse', 'Moselle', 'Bas-Rhin', 'Haut-Rhin', 'Vosges'],
    'Hauts-de-France': ['Aisne', 'Nord', 'Oise', 'Pas-de-Calais', 'Somme'],
    'Île-de-France': ['Paris', 'Seine-et-Marne', 'Yvelines', 'Essonne', 'Hauts-de-Seine', 'Seine-Saint-Denis', 'Val-de-Marne', 'Val-d\'Oise'],
    'Normandie': ['Calvados', 'Eure', 'Manche', 'Orne', 'Seine-Maritime'],
    'Nouvelle-Aquitaine': ['Charente', 'Charente-Maritime', 'Corrèze', 'Creuse', 'Dordogne', 'Gironde', 'Landes', 'Lot-et-Garonne', 'Pyrénées-Atlantiques', 'Deux-Sèvres', 'Vienne', 'Haute-Vienne'],
    'Occitanie': ['Ariège', 'Aude', 'Aveyron', 'Gard', 'Haute-Garonne', 'Gers', 'Hérault', 'Lot', 'Lozère', 'Hautes-Pyrénées', 'Pyrénées-Orientales', 'Tarn', 'Tarn-et-Garonne'],
    'Pays de la Loire': ['Loire-Atlantique', 'Maine-et-Loire', 'Mayenne', 'Sarthe', 'Vendée'],
    'Provence-Alpes-Côte d\'Azur': ['Alpes-de-Haute-Provence', 'Hautes-Alpes', 'Alpes-Maritimes', 'Bouches-du-Rhône', 'Var', 'Vaucluse']
}

Regions = [
    'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    # 'Corse',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]

Name2num = {
    'Ain': '01',
    'Aisne': '02',
    'Allier': '03',
    'Alpes-de-Haute-Provence': '04',
    'Hautes-Alpes': '05',
    'Alpes-Maritimes': '06',
    'Ardèche': '07',
    'Ardennes': '08',
    'Ariège': '09',
    'Aube': '10',
    'Aude': '11',
    'Aveyron': '12',
    'Bouches-du-Rhône': '13',
    'Calvados': '14',
    'Cantal': '15',
    'Charente': '16',
    'Charente-Maritime': '17',
    'Cher': '18',
    'Corrèze': '19',
    'Corse-du-Sud': '2A',
    'Haute-Corse': '2B',
    'Côte-d\'Or': '21',
    'Côtes-d\'Armor': '22',
    'Creuse': '23',
    'Dordogne': '24',
    'Doubs': '25',
    'Drôme': '26',
    'Eure': '27',
    'Eure-et-Loir': '28',
    'Finistère': '29',
    'Gard': '30',
    'Haute-Garonne': '31',
    'Gers': '32',
    'Gironde': '33',
    'Hérault': '34',
    'Ille-et-Vilaine': '35',
    'Indre': '36',
    'Indre-et-Loire': '37',
    'Isère': '38',
    'Jura': '39',
    'Landes': '40',
    'Loir-et-Cher': '41',
    'Loire': '42',
    'Haute-Loire': '43',
    'Loire-Atlantique': '44',
    'Loiret': '45',
    'Lot': '46',
    'Lot-et-Garonne': '47',
    'Lozère': '48',
    'Maine-et-Loire': '49',
    'Manche': '50',
    'Marne': '51',
    'Haute-Marne': '52',
    'Mayenne': '53',
    'Meurthe-et-Moselle': '54',
    'Meuse': '55',
    'Morbihan': '56',
    'Moselle': '57',
    'Nièvre': '58',
    'Nord': '59',
    'Oise': '60',
    'Orne': '61',
    'Pas-de-Calais': '62',
    'Puy-de-Dôme': '63',
    'Pyrénées-Atlantiques': '64',
    'Hautes-Pyrénées': '65',
    'Pyrénées-Orientales': '66',
    'Bas-Rhin': '67',
    'Haut-Rhin': '68',
    'Rhône': '69',
    'Haute-Saône': '70',
    'Saône-et-Loire': '71',
    'Sarthe': '72',
    'Savoie': '73',
    'Haute-Savoie': '74',
    'Paris': '75',
    'Seine-Maritime': '76',
    'Seine-et-Marne': '77',
    'Yvelines': '78',
    'Deux-Sèvres': '79',
    'Somme': '80',
    'Tarn': '81',
    'Tarn-et-Garonne': '82',
    'Var': '83',
    'Vaucluse': '84',
    'Vendée': '85',
    'Vienne': '86',
    'Haute-Vienne': '87',
    'Vosges': '88',
    'Yonne': '89',
    'Territoire de Belfort': '90',
    'Essonne': '91',
    'Hauts-de-Seine': '92',
    'Seine-Saint-Denis': '93',
    'Val-de-Marne': '94',
    'Val-d\'Oise': '95'
}


AREA = []
for area in CFG.area:
    AREA.extend(Administratives[area] )
AREA

C:\Users\demo\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


['Ain',
 'Allier',
 'Ardèche',
 'Cantal',
 'Drôme',
 'Isère',
 'Loire',
 'Haute-Loire',
 'Puy-de-Dôme',
 'Rhône',
 'Savoie',
 'Haute-Savoie',
 "Côte-d'Or",
 'Doubs',
 'Jura',
 'Nièvre',
 'Haute-Saône',
 'Saône-et-Loire',
 'Yonne',
 'Territoire de Belfort',
 "Côtes-d'Armor",
 'Finistère',
 'Ille-et-Vilaine',
 'Morbihan',
 'Cher',
 'Eure-et-Loir',
 'Indre',
 'Indre-et-Loire',
 'Loir-et-Cher',
 'Loiret',
 'Ardennes',
 'Aube',
 'Marne',
 'Haute-Marne',
 'Meurthe-et-Moselle',
 'Meuse',
 'Moselle',
 'Bas-Rhin',
 'Haut-Rhin',
 'Vosges',
 'Aisne',
 'Nord',
 'Oise',
 'Pas-de-Calais',
 'Somme',
 'Paris',
 'Seine-et-Marne',
 'Yvelines',
 'Essonne',
 'Hauts-de-Seine',
 'Seine-Saint-Denis',
 'Val-de-Marne',
 "Val-d'Oise",
 'Calvados',
 'Eure',
 'Manche',
 'Orne',
 'Seine-Maritime',
 'Charente',
 'Charente-Maritime',
 'Corrèze',
 'Creuse',
 'Dordogne',
 'Gironde',
 'Landes',
 'Lot-et-Garonne',
 'Pyrénées-Atlantiques',
 'Deux-Sèvres',
 'Vienne',
 'Haute-Vienne',
 'Ariège',
 'Aude',
 'Aveyron',
 'Gard

# Data tourisme 

    - le dataset provient de data.gouv.fr
    - Via API possibilité d'avoir une extraction actualisée de la base ( frequence journalière)

In [2]:
data_path = './osm_datas'
file_name = 'datatourisme-place-20230529'

# tourism = gpd.read_file(os.path.join(data_path,file_name + '.csv'))
# tourism.to_feather(os.path.join(data_path,file_name + '.feather'))



tourism = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))
display(tourism.info())
tourism.shape

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 338032 entries, 0 to 338031
Data columns (total 17 columns):
 #   Column                                    Non-Null Count   Dtype   
---  ------                                    --------------   -----   
 0   Nom_du_POI                                338032 non-null  object  
 1   Categories_de_POI                         338032 non-null  object  
 2   Latitude                                  338032 non-null  object  
 3   Longitude                                 338032 non-null  object  
 4   Adresse_postale                           338032 non-null  object  
 5   Code_postal_et_commune                    338032 non-null  object  
 6   Covid19_mesures_specifiques               338032 non-null  object  
 7   Covid19_est_en_activite                   338032 non-null  object  
 8   Covid19_periodes_d_ouvertures_confirmees  338032 non-null  object  
 9   Createur_de_la_donnee                     338032 non-null  object  
 10  

None

(338032, 17)

In [3]:
tourism.sample(20)

,Nom_du_POI,Categories_de_POI,Latitude,Longitude,Adresse_postale,Code_postal_et_commune,Covid19_mesures_specifiques,Covid19_est_en_activite,Covid19_periodes_d_ouvertures_confirmees,Createur_de_la_donnee,SIT_diffuseur,Date_de_mise_a_jour,Contacts_du_POI,Classements_du_POI,Description,URI_ID_du_POI,geometry
221887,Auberge les olivades,https://www.datatourisme.fr/ontology/core#Plac...,43.71955,5.244375,RN 7 Le petit Paris,13370#Mallemort,,,,Provence Tourisme,Provence Tourisme,2023-02-08,#+33 4 90 57 41 31#aubergelesolivades@wanadoo....,,4 chambres et 3 appartements idéalement situés...,https://data.datatourisme.fr/58/462053c6-7f1d-...,None
208174,Église Saint-Hilaire de Péreuil,https://www.datatourisme.fr/ontology/core#Cult...,45.481442,-0.022405,Péreuil 3 Place Roger Vincent,16250#Val des Vignes,,,,Office de tourisme du Sud-Charente,Charentes Tourisme - source : Apidae Tourisme,2023-03-23,#+33 5 45 66 37 17#mairie@valdesvignes.fr#http...,,L'église paroissiale Saint-Hilaire est un ense...,https://data.datatourisme.fr/13/eb3ad441-3e64-...,None
34211,Au gîte de la Germanette - Maison de vacances,https://www.datatourisme.fr/ontology/core#Plac...,44.405632,5.728119,Base de loisirs de la Germanette Bas Isclamare,05700#Serres,,,,Office de Tourisme Sisteron Buëch,Apidae Tourisme Scic SA - source : Apidae Tour...,2023-01-05,#+33 6 86 76 24 48#manu.danjou@gmail.com#http:...,Accueil Vélo#France Vélo Tourisme,"Entre ""Mer"" et Montagne, le gîte est idéalemen...",https://data.datatourisme.fr/13/34e95dac-9675-...,None
76797,JARDIN DE LA PLANTADE ET MOULINS CORDIER,https://www.datatourisme.fr/ontology/core#Cult...,43.3446317,3.20776631,37 Route de Murviel,34500#Béziers,,,,OT BEZIERS MEDITERRANEE,Hérault Tourisme,2023-05-28,JARDIN DE LA PLANTADE ET MOULINS CORDIER#+33 4...,,"Au bord de l'Orb , ce jardin ombragé est parti...",https://data.datatourisme.fr/32/7e1338ff-b361-...,None
43566,Tiki III,https://www.datatourisme.fr/ontology/core#Plac...,43.450779,4.398812,D38 - Lieu dit le Tiki,13460#Saintes-Maries-de-la-Mer,,,,Office de Tourisme des Saintes Maries de la Mer,Provence Tourisme / Office de Tourisme des Sai...,2023-02-07,#+33 4 90 97 81 68#contact@tiki3.fr#http://www...,Accueil Vélo#France Vélo Tourisme,"A bord du TIKI III, bateau typique à roue, déc...",https://data.datatourisme.fr/58/a7d678c6-c10b-...,None
158468,P'TITS LÉGUMES & CIE,https://www.datatourisme.fr/ontology/core#Plac...,49.4670395330305,6.31126721117233,GANDREN 33 Bis rue de la Forêt,57570#Beyren-lès-Sierck,,,,OT CATTENOM ET ENVIRONS,Système d’information touristique Lorrain,2023-05-08,#+33 3 82 83 61 25#pascale.cannata@orange.fr#h...,,"Des légumes, des fruits mais aussi des fleurs ...",https://data.datatourisme.fr/10/820a9e28-57c6-...,None
245894,Magnifique T2 rénové et modernisé,https://www.datatourisme.fr/ontology/core#Plac...,43.311253,6.628662,29 avenue Georges Pompidou Le Surcouf App C103,83120#Sainte-Maxime,,,,Office de tourisme de Sainte Maxime,Office de tourisme de Sainte Maxime - source :...,2023-04-21,#+33 6 09 36 41 75##,,"Magnifique T2 rénové, moderne dans résidence d...",https://data.datatourisme.fr/13/4a397bba-0778-...,None
141454,Le Barbican,https://www.datatourisme.fr/ontology/core#Food...,49.7603922,0.36765213,97 quai Bérigny,76400#Fécamp,,,,Normandie Tourisme / Seine-Maritime Attractivité,Normandie Tourisme,2023-05-28,#+33 2 27 30 59 95##https://www.lebarbican.fr/...,,"Au Barbican, le chef travaille uniquement avec...",https://data.datatourisme.fr/30/ba35a8f5-6ff9-...,None
110816,Cash Piscine,https://www.datatourisme.fr/ontology/core#Plac...,44.361552,4.695618,avenue Auguste Rodin,26700#Pierrelatte,,,,Office de Tourisme Drôme Sud Provence,Agence d'Attractivité de la Drôme - source : A...,2018-08-02,#+33 4 75 50 26 94##,,vente de piscines.,https://data.datatourisme.fr/13/174dc41a-dacc-...,None
32322,Les Jardins d'Epicure,https://www.datatourisme.fr/ontology/core#Food...,49.139171,1.657087,16 Grande Rue,95710#Bray-et-Lû,,,,Parc naturel régional du Vexin França

In [4]:
tourism.loc[0,'Categories_de_POI']

'https://www.datatourisme.fr/ontology/core#PlaceOfInterest|https://www.datatourisme.fr/ontology/core#PointOfInterest|https://www.datatourisme.fr/ontology/core#Producer|https://www.datatourisme.fr/ontology/core#TastingProvider|http://schema.org/LocalBusiness'

In [5]:
tag = tourism.loc[0,'Categories_de_POI']


tag.split('https://www.datatourisme.fr/ontology/core#')[-1].split('|')[0]

'TastingProvider'

In [6]:
from shapely import Point

tourism['tag'] = tourism['Categories_de_POI'].apply(
    lambda s : s.split('https://www.datatourisme.fr/ontology/core#')[-1].split('|')[0]
)

for n, (x, y)  in enumerate(zip(tourism.Latitude, tourism.Longitude)):
    tourism.loc[n,'geometry'] = Point(y,x)

In [7]:

display(len(tourism.tag.unique()))
tourism.tag.unique()

326

array(['TastingProvider', 'Store', 'WineCellar', 'ZooAnimalPark',
       'SportsAndLeisurePlace', 'PointOfInterest', 'Traineeship',
       'Restaurant', 'FastFoodRestaurant', 'BusinessPlace',
       'SelfServiceCafeteria', 'SightseeingBoat', 'TouristTrain',
       'Practice', 'GourmetRestaurant', 'MedicalPlace', 'RemembranceSite',
       'TechnicalHeritage', 'ServiceProvider', 'PicnicArea',
       'SelfCateringAccommodation', 'YachtClub', 'Marina', 'Trader',
       'TaxiCompany', 'RentalAccommodation', 'TeachingFarm',
       'CamperVanArea', 'TrackRollerOrSkateBoard', 'Gymnasium',
       'NonHousingRealEstateRental', 'TennisComplex', 'Trampoline',
       'NaturalHeritage', 'FarmhouseInn', 'Tasting', 'Transporter',
       'HolidayResort', 'Spa', 'HotelTrade', 'BarOrPub',
       'InterpretationCentre', 'Visit', 'GroupLodging', 'SwimmingPool',
       'DefenceSite', 'BrasserieOrTavern', 'Accommodation', 'Transport',
       'CampingAndCaravanning', 'TouristBus', 'NauticalCentre',
       'Co

---------------------

Vérification des tags sur courbevoie 92026 et puteaux 92062
suite à valeurs abherantes aprés classif

In [9]:
tourism.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 338032 entries, 0 to 338031
Data columns (total 18 columns):
 #   Column                                    Non-Null Count   Dtype   
---  ------                                    --------------   -----   
 0   Nom_du_POI                                338032 non-null  object  
 1   Categories_de_POI                         338032 non-null  object  
 2   Latitude                                  338032 non-null  object  
 3   Longitude                                 338032 non-null  object  
 4   Adresse_postale                           338032 non-null  object  
 5   Code_postal_et_commune                    338032 non-null  object  
 6   Covid19_mesures_specifiques               338032 non-null  object  
 7   Covid19_est_en_activite                   338032 non-null  object  
 8   Covid19_periodes_d_ouvertures_confirmees  338032 non-null  object  
 9   Createur_de_la_donnee                     338032 non-null  object  
 10  

In [12]:
tourism.query(" Code_postal_et_commune.str.contains('92400') or  Code_postal_et_commune.str.contains('92800') ")

,Nom_du_POI,Categories_de_POI,Latitude,Longitude,Adresse_postale,Code_postal_et_commune,Covid19_mesures_specifiques,Covid19_est_en_activite,Covid19_periodes_d_ouvertures_confirmees,Createur_de_la_donnee,SIT_diffuseur,Date_de_mise_a_jour,Contacts_du_POI,Classements_du_POI,Description,URI_ID_du_POI,geometry,tag
7373,Adagio City Aparthotel Kleber La Défense,https://www.datatourisme.fr/ontology/core#Plac...,48.8942177,2.2486810000000332,73 avenue Gambetta,92400#Courbevoie,,,,Office de Tourisme de Paris,Office de Tourisme de Paris,2022-07-11,#+33 1 56 37 20 00#H6788@adagio-city.com#http:...,4 étoiles#Classement officiel des hébergements...,Cette résidence de tourisme est installée sur ...,https://data.datatourisme.fr/19/7dc099ed-4782-...,POINT (2.24868 48.89422),HolidayResort
8148,Adagio La Défense Le Parc,https://www.datatourisme.fr/ontology/core#Plac...,48.8971723,2.2363632999999936,1 rue des Lilas d'Espagne,92400#Courbevoie,,,,Office de Tourisme de Paris,Office de Tourisme de Paris,2022-07-11,#+33 1 55 70 79 79#h6787@adagio-city.com#http:...,4 étoiles#Classement officiel des hébergements...,Cette résidence de tourisme est installée dans...,https://data.datatourisme.fr/19/d15c4bb8-a603-...,POINT (2.23636 48.89717),HolidayResort
8401,Adagio access La Défense Léonard de Vinci,https://www.datatourisme.fr/ontology/core#Plac...,48.8968415,2.2346697000000404,88 rue des Étudiants,92400#Courbevoie,,,,Office de Tourisme de Paris,Office de Tourisme de Paris,2021-02-16,#+33 1 41 88 20 00#H8363@adagio-city.com#http:...,4 étoiles#Classement officiel des hébergements...,Cette résidence de tourisme se situe face au p...,https://data.datatourisme.fr/19/98223472-d0e5-...,POINT (2.23467 48.89684),HolidayResort
9577,Adagio La Défense Puteaux,https://www.datatourisme.fr/ontology/core#Plac...,48.8847469,2.24424190000002,70 rue Roque de Fillol,92800#Puteaux,,,,Office de Tourisme de Paris,Office de Tourisme de Paris,2021-02-16,#+33 1 55 62 12 00#h8987@adagio-city.com#https...,3 étoiles#Classement officiel des hébergements...,Cette résidence de tourisme est située dans le...,https://data.datatourisme.fr/19/02498904-2d48-...,POINT (2.24424 48.88475),HolidayResort
11075,Fraser Suites Harmonie Paris La Défense,https://www.datatourisme.fr/ontology/core#Plac...,48.8892055,2.25083989999996,6 boulevard de Neuilly,92400#Courbevoie,,,,Office de Tourisme de Paris,Office de Tourisme de Paris,2022-11-23,#+33 1 55 23 26 00#sales.paris@frasershospital...,5 étoiles#Classement officiel des hébergements...,Cette résidence de tourisme accueille sa clien...,https://data.datatourisme.fr/19/eaf010cb-52b8-...,POINT (2.25084 48.88921),HolidayResort
11367,Adagio La Défense Courbevoie,https://www.datatourisme.fr/ontology/core#Plac...,48.8933593,2.253148900000042,2 - 4 Place des Pléiades,92400#Courbevoie,,,,Office de Tourisme de Paris,Office de Tourisme de Paris,2021-02-16,#+33 1 46 40 85 20#h9297@adagio-city.com#https...,4 étoiles#Classement officiel des hébergements...,Cette résidence se situe tout près du bowling ...,https://data.datatourisme.fr/19/ab725ad5-a62f-...,POINT (2.25315 48.89336),HolidayResort
32723,Ibis Paris La Défense Centre,https://www.datatourisme.fr/ontology/core#Plac...,48.88845269999999,2.250994300000002,4 boulevard de Neuilly,92400#Courbevoie,,,,Office de Tourisme de Paris,Office de Tourisme de Paris,2022-07-11,#+33 1 41 97 40 40#H0771@accor.com#https://ibi...,3 étoiles#Classement officiel des hébergements...,Implanté au cœur du quartier des affaires de L...,https://data.datatourisme.fr/19/6ddb9f39-550b-...,POINT (2.25099 48.88845),HotelTrade
53886,Meliá Paris La Défense,https://www.datatourisme.fr/ontology/core#Plac...,48.8884273,2.248811299999943,2-4 esplanade du Général de Gaulle,92400#Courbevoie,,,,Office de Tourisme de Paris,Office de Tourisme de Paris,2022-07-11,#+33 1 75 57 99 00#reservations.ladefense@meli...,4 étoiles#Classement officiel des hébergements...,,https://data.datatourisme.fr/19/381d2b06-5661-...,POINT (2.24881 48.88843),HotelTrade
5389

-------------------------

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification")

def transform_tags(tags, labels, output=True):
    tag2class = {}

    class2tag = {}
    for key in labels:
        class2tag[key] = []

    total = len(tags)
    for n,key in enumerate(tags):
        sentence = ' '.join(key.split(';'))

        # result = classifier(translator(sentence)[0]['translation_text'], candidate_labels, multi_label=True)['labels'][0]
        result = classifier(sentence, labels, multi_label=True)['labels'][0]
        
        # display(result)
        tag2class[sentence] = result
        class2tag[result].append(sentence)
        if output:
            print(f'{sentence} ->[ {tag2class[sentence]} ]', end=' ' )
        print(f'| {n+1}/{total} ~ {(n+1) / total:.2%}', end=' '*50+'\r')          

    return     tag2class, class2tag    


# Classification des tags

    - Ajouts et afinage des labels ... pas trouvé de méthode autre que rajouter des items au vu des resultats

In [ ]:
candidate_labels = [
    'leisure center, therapy center, outdoor activity, recreation, trail, hikiing, tour, nature, harbor, farm, animal, bicycle charge point',
    'monument, historic site, church, monastery, castle, ruins, hut, palace, museum',
    'food, restaurant , bar, cofee, icecream',
    'hotel, bed, housse rental, vacation rental, campsite',
    'vehicule, parking, vehicule rental,',
    'business, healtcare, office, institution, official building, work, factory, facility',
    'Electric vehicule Charging Point'                   # <- Voir si les bornes sont identiques au fichier consolidé sur la base
]

labels2legend=dict(zip(
    candidate_labels,
    [
        'activity',
        'visit',
        'fooding',
        'sleeping',
        'vehicule',
        'working',
        'EV Infra'
    ]
))

joblib.dump(candidate_labels, os.path.join(CFG.data_path,f'[{CFG.area_name}]candidate_labels.pkl'))
joblib.dump(labels2legend, os.path.join(CFG.data_path,f'[{CFG.area_name}]labels2legend.pkl'))

tag2class, class2tag = transform_tags(tourism.tag.unique(), candidate_labels, output=False)
display('tag to classes:',tag2class)
display('Class content:', class2tag)   

In [ ]:
tourism['Class'] = tourism.tag.map(tag2class)#.map(labels2legend)

In [ ]:
tourism

In [ ]:
from shapely import box

# box(-10,40,10,60)

metropolitan_tourism = tourism.clip(box(-10,40,8.5,60))

metropolitan_tourism.plot(
    figsize=(60,60), 
    column = metropolitan_tourism.Class.values,
    cmap = 'nipy_spectral',
    categorical= True,
    legend =True,
    legend_kwds={'fontsize':30,
                 'loc':'upper left',
                 'shadow':False,
                 'title': 'Activities',
                 'title_fontsize':50,
                 'markerscale':2,
                 'framealpha':.05},
    markersize = 12,
    alpha=0.55)

In [ ]:
data_path = './osm_datas'
file_name = 'datatourisme-place-20230529-tagged'

# tourism = gpd.read_file(os.path.join(data_path,file_name + '.csv'))
# tourism.to_feather(os.path.join(data_path,file_name + '.feather'))



tourism.to_feather(os.path.join(data_path,file_name + '.feather'))

: 

In [ ]:
tourism = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))

In [ ]:
tourism.info()

In [ ]:
for n, (x, y)  in enumerate(zip(tourism.Longitude, tourism.Latitude)):
    tourism.loc[n,'geometry'] = Point(x,y)